In [1]:
%pwd

'd:\\VS code Projects\\AI-Powered Data cleaning Agent\\AI-Powered-Data-Cleaning-Agent\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\VS code Projects\\AI-Powered Data cleaning Agent\\AI-Powered-Data-Cleaning-Agent'

In [ ]:
# Step 1 - Import Libraries
import pandas as pd
import numpy as np


# Exploring the Dataset

In [ ]:
# Step 2 - Load the dataset
df = pd.read_csv("data/raw/dirty_cafe_sales.csv")
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


Shape of dataset: (10000, 8)


In [14]:
# Step 3 - Basic Inspection
print("shape of dataset:", df.shape)
print("\nFirst 5 rows:")
display(df.head())

shape of dataset: (10000, 8)

First 5 rows:


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [15]:
print("\nDataset Info:")
print(df.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
None


In [16]:
print("\nMissing values count per column:")
print(df.isnull().sum())


Missing values count per column:
Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64


In [17]:
print("\nDuplicate rows count:", df.duplicated().sum())


Duplicate rows count: 0


# Data quality Detection

In [18]:
# Missing Data
print("---- Missing Data ----")
missing_counts = df.isnull().sum()
print(missing_counts[missing_counts > 0].sort_values(ascending=False))
print()

---- Missing Data ----
Location            3265
Payment Method      2579
Item                 333
Price Per Unit       179
Total Spent          173
Transaction Date     159
Quantity             138
dtype: int64



In [19]:
# Duplicate Rows
print("---- Duplicate Rows ----")
print(f"Exact duplicate rows: {df.duplicated().sum()}")
print()

---- Duplicate Rows ----
Exact duplicate rows: 0



In [24]:
import pandas as pd
import warnings

print("---- Data Type Issues ----")
for col in df.columns:
    if df[col].dtype == 'object':
        # Check numeric
        if pd.to_numeric(df[col], errors='coerce').notna().all():
            print(f"Possible numeric stored as string: {col}")

        # Check date (suppress warnings during test)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            date_series = pd.to_datetime(df[col], errors='coerce', format=None)
        if date_series.notna().all():
            print(f"Possible date stored as string: {col}")
print()


---- Data Type Issues ----



In [26]:
# 4️⃣ Inconsistent Formatting (Text Columns)
print("---- Inconsistent Formatting ----")
for col in df.select_dtypes(include='object'):
    sample_values = df[col].dropna().unique()[:10]
    print(f"{col} sample values: {sample_values}")
print()

---- Inconsistent Formatting ----
Transaction ID sample values: ['TXN_1961373' 'TXN_4977031' 'TXN_4271903' 'TXN_7034554' 'TXN_3160411'
 'TXN_2602893' 'TXN_4433211' 'TXN_6699534' 'TXN_4717867' 'TXN_2064365']
Item sample values: ['Coffee' 'Cake' 'Cookie' 'Salad' 'Smoothie' 'UNKNOWN' 'Sandwich' 'ERROR'
 'Juice' 'Tea']
Quantity sample values: ['2' '4' '5' '3' '1' 'ERROR' 'UNKNOWN']
Price Per Unit sample values: ['2.0' '3.0' '1.0' '5.0' '4.0' '1.5' 'ERROR' 'UNKNOWN']
Total Spent sample values: ['4.0' '12.0' 'ERROR' '10.0' '20.0' '9.0' '16.0' '15.0' '25.0' '8.0']
Payment Method sample values: ['Credit Card' 'Cash' 'UNKNOWN' 'Digital Wallet' 'ERROR']
Location sample values: ['Takeaway' 'In-store' 'UNKNOWN' 'ERROR']
Transaction Date sample values: ['2023-09-08' '2023-05-16' '2023-07-19' '2023-04-27' '2023-06-11'
 '2023-03-31' '2023-10-06' '2023-10-28' '2023-07-28' '2023-12-31']



In [27]:
# 5️⃣ Outliers (Numerical)
print("---- Outliers (Numerical) ----")
numeric_cols = df.select_dtypes(include=np.number).columns
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    if not outliers.empty:
        print(f"{col}: {len(outliers)} outliers detected")
print()

---- Outliers (Numerical) ----



In [28]:
# 6️⃣ Categorical Value Issues
print("---- Categorical Value Issues ----")
for col in df.select_dtypes(include='object'):
    value_counts = df[col].value_counts()
    rare_values = value_counts[value_counts < 3]
    if not rare_values.empty:
        print(f"{col}: Rare categories -> {list(rare_values.index)}")
print()

---- Categorical Value Issues ----
Transaction ID: Rare categories -> ['TXN_9226047', 'TXN_8567525', 'TXN_4583012', 'TXN_6796890', 'TXN_9933628', 'TXN_4302199', 'TXN_5548914', 'TXN_3528020', 'TXN_9668108', 'TXN_8076061', 'TXN_7936002', 'TXN_3124078', 'TXN_6120851', 'TXN_5762440', 'TXN_9954652', 'TXN_8866974', 'TXN_8927252', 'TXN_1736287', 'TXN_7640952', 'TXN_8467949', 'TXN_5695074', 'TXN_5183041', 'TXN_7958992', 'TXN_9400181', 'TXN_2616390', 'TXN_5132361', 'TXN_3567645', 'TXN_3522028', 'TXN_3709394', 'TXN_8876618', 'TXN_6769710', 'TXN_3765707', 'TXN_9499313', 'TXN_8078640', 'TXN_5455792', 'TXN_1491578', 'TXN_9620080', 'TXN_6650263', 'TXN_2427584', 'TXN_2083138', 'TXN_6688524', 'TXN_2655815', 'TXN_1080432', 'TXN_6855453', 'TXN_9130559', 'TXN_8853997', 'TXN_7710508', 'TXN_9677376', 'TXN_9099694', 'TXN_2537617', 'TXN_8051289', 'TXN_9023317', 'TXN_8813311', 'TXN_6421134', 'TXN_2080895', 'TXN_3578141', 'TXN_5522862', 'TXN_8614868', 'TXN_3363746', 'TXN_8914892', 'TXN_6342161', 'TXN_7742742',

In [29]:
# 7️⃣ Business Logic Violations
print("---- Business Logic Violations ----")
if 'Quantity' in df.columns and 'UnitPrice' in df.columns:
    negative_qty = df[df['Quantity'] < 0]
    if not negative_qty.empty:
        print(f"Negative quantities: {len(negative_qty)} rows")
    negative_price = df[df['UnitPrice'] < 0]
    if not negative_price.empty:
        print(f"Negative prices: {len(negative_price)} rows")
    if 'Total' in df.columns:
        mismatch_total = df[(df['Quantity'] * df['UnitPrice']) != df['Total']]
        print(f"Total column mismatches: {len(mismatch_total)} rows")
print("---- Detection Complete ----")

---- Business Logic Violations ----
---- Detection Complete ----


# Now we are Cleaning the Data 

In [ ]:
# Step 3.1 - Missing Data Cleaning 

# 1️⃣ Calculate missing percentage
missing_percent = (df.isnull().sum() / len(df)) * 100
print(missing_percent)

# 2️⃣ Handle high-missing columns (>40%)
high_missing_cols = missing_percent[missing_percent > 40].index
print(f"Columns with high missing %: {list(high_missing_cols)}")

# 3️⃣ Fill categorical columns
for col in ['Location', 'Payment Method']:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].mode()[0])

# 4️⃣ Convert numerical columns from string to float before filling
for col in ['Price Per Unit', 'Total Spent', 'Quantity']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to float, NaN if invalid
        df[col] = df[col].fillna(df[col].median())  # Fill missing with median

# 5️⃣ Fill date column
if 'Transaction Date' in df.columns:
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
    median_date = df['Transaction Date'].median()
    df['Transaction Date'] = df['Transaction Date'].fillna(median_date)

print("✅ Missing data handled with type conversion.")


Transaction ID      0.00
Item                3.33
Quantity            1.38
Price Per Unit      1.79
Total Spent         1.73
Payment Method      0.00
Location            0.00
Transaction Date    1.59
dtype: float64
Columns with high missing %: []
✅ Missing data handled with type conversion.


In [32]:
# Step 3.2 - Duplicate Row Handling

# Count duplicates (excluding the index)
duplicate_count = df.duplicated().sum()
print(f"🔍 Found {duplicate_count} duplicate rows.")

# Remove duplicates
df = df.drop_duplicates()
print(f"✅ Removed duplicates. New shape: {df.shape}")


🔍 Found 0 duplicate rows.
✅ Removed duplicates. New shape: (10000, 8)


In [33]:
# Step 3.3 - Data Type & Format Cleaning

# Ensure numeric columns are floats
numeric_cols = ['Quantity', 'Price Per Unit', 'Total Spent']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Ensure date column is datetime
if 'Transaction Date' in df.columns:
    df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

# Standardize categorical text columns
categorical_cols = ['Item', 'Payment Method', 'Location']
for col in categorical_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.title()

print("✅ Data types and formats standardized.")
print(df.dtypes)


✅ Data types and formats standardized.
Transaction ID              object
Item                        object
Quantity                   float64
Price Per Unit             float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object


In [34]:
# Step 3.4 - Outlier Detection & Handling

# Function to find outliers using IQR method
def find_outliers_iqr(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[col] < lower_bound) | (data[col] > upper_bound)]

outlier_summary = {}

for col in ['Quantity', 'Price Per Unit', 'Total Spent']:
    if col in df.columns:
        outliers = find_outliers_iqr(df, col)
        outlier_summary[col] = len(outliers)
        print(f"🔍 {col}: Found {len(outliers)} outliers")

# Optionally: Remove outliers
for col in ['Quantity', 'Price Per Unit', 'Total Spent']:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

print(f"✅ Outliers removed. New shape: {df.shape}")


🔍 Quantity: Found 0 outliers
🔍 Price Per Unit: Found 0 outliers
🔍 Total Spent: Found 259 outliers
✅ Outliers removed. New shape: (9741, 8)


In [35]:
# Step 3.5 - Detect & Fix Inconsistent Values

# Function to show unique values sorted for manual inspection
def show_inconsistencies(col):
    print(f"\n🔍 Unique values in '{col}':")
    print(sorted(df[col].unique()))

# Check for inconsistencies in categorical columns
categorical_cols = ['Item', 'Payment Method', 'Location']
for col in categorical_cols:
    if col in df.columns:
        show_inconsistencies(col)

# Example fix: Standardize Payment Method spelling
if 'Payment Method' in df.columns:
    df['Payment Method'] = df['Payment Method'].str.strip().str.title()

# Example fix: Standardize Location spacing & case
if 'Location' in df.columns:
    df['Location'] = df['Location'].str.strip().str.title()

print("\n✅ Inconsistencies partially fixed. Manual mapping may still be needed for some cases.")



🔍 Unique values in 'Item':
['Cake', 'Coffee', 'Cookie', 'Error', 'Juice', 'Nan', 'Salad', 'Sandwich', 'Smoothie', 'Tea', 'Unknown']

🔍 Unique values in 'Payment Method':
['Cash', 'Credit Card', 'Digital Wallet', 'Error', 'Unknown']

🔍 Unique values in 'Location':
['Error', 'In-Store', 'Takeaway', 'Unknown']

✅ Inconsistencies partially fixed. Manual mapping may still be needed for some cases.


In [37]:
import numpy as np

# Step 3.6 - Rule-based mapping for invalid categorical values

# Define mappings for invalid entries
invalid_values = ["Error", "Unknown", "Nan", "nan", "NaN", None]

# Replace invalid entries with NaN in categorical columns
for col in ['Item', 'Payment Method', 'Location']:
    df[col] = df[col].replace(invalid_values, np.nan)

# Optional: fill missing categories with most frequent value
for col in ['Item', 'Payment Method', 'Location']:
    if df[col].isna().sum() > 0:
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value)

print("✅ Invalid values replaced and missing categories filled.")
print(df[['Item', 'Payment Method', 'Location']].head(10))


✅ Invalid values replaced and missing categories filled.
       Item  Payment Method  Location
0    Coffee     Credit Card  Takeaway
1      Cake            Cash  In-Store
2    Cookie     Credit Card  In-Store
3     Salad  Digital Wallet  Takeaway
4    Coffee  Digital Wallet  In-Store
5  Smoothie     Credit Card  Takeaway
6     Juice  Digital Wallet  Takeaway
7  Sandwich            Cash  Takeaway
8     Juice  Digital Wallet  Takeaway
9  Sandwich  Digital Wallet  In-Store


In [38]:
# Step 4 - Data Quality Report

def data_quality_report(df):
    report = {}
    
    for col in df.columns:
        report[col] = {
            'dtype': str(df[col].dtype),
            'missing_count': df[col].isna().sum(),
            'missing_percent': round((df[col].isna().sum() / len(df)) * 100, 2),
            'unique_values': df[col].nunique()
        }
    return pd.DataFrame(report).T

final_report = data_quality_report(df)

print("📊 Final Data Quality Report")
display(final_report)


📊 Final Data Quality Report


,dtype,missing_count,missing_percent,unique_values
Transaction ID,object,0,0.0,9741
Item,object,0,0.0,8
Quantity,float64,0,0.0,5
Price Per Unit,float64,0,0.0,6
Total Spent,float64,0,0.0,16
Payment Method,object,0,0.0,3
Location,object,0,0.0,2
Transaction Date,datetime64[ns],0,0.0,365


In [41]:
import os
from datetime import datetime

# Create processed folder if it doesn't exist
os.makedirs("data/processed", exist_ok=True)

# Create a timestamp string
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# File path with timestamp
output_path = f"data/processed/cafe_sales_cleaned_{timestamp}.csv"

# Save the cleaned data
df.to_csv(output_path, index=False)

print(f"✅ Cleaned data saved to: {output_path}")


✅ Cleaned data saved to: data/processed/cafe_sales_cleaned_20250809_121006.csv


Each time you run the save code with a timestamp, it makes a new file instead of overwriting the old one.

## The timestamp is there so:

- Versioning → You can keep track of when each cleaned dataset was created.
Example: If today’s cleaning works but tomorrow you accidentally break something, you still have yesterday’s clean file.

- Traceability → In real AI pipelines, you want to know exactly which cleaned dataset was used for model training — the timestamp in the filename makes that possible.

- Safety → It avoids overwriting your previous work by mistake.

In [44]:
import os
from datetime import datetime
import pandas as pd

# Paths
processed_dir = "data/processed"

# 1️⃣ Load the most recent processed file
processed_files = sorted(os.listdir(processed_dir), reverse=True)
latest_processed = os.path.join(processed_dir, processed_files[0])
df_clean = pd.read_csv(latest_processed)

# 2️⃣ Manually input old stats from earlier cleaning steps
raw_shape = (10000, 8)  # Replace with actual raw shape from earlier
processed_shape = df_clean.shape
missing_before = {
    'Transaction ID': 0.00,
    'Item': 3.33,
    'Quantity': 1.38,
    'Price Per Unit': 1.79,
    'Total Spent': 1.73,
    'Payment Method': 0.00,
    'Location': 0.00,
    'Transaction Date': 1.59
}  # Replace with earlier missing value percentages/counts
missing_after = df_clean.isnull().sum()

# 3️⃣ Create report
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
report_lines = []
report_lines.append("=== Data Cleaning Report ===\n")
report_lines.append(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
report_lines.append(f"Raw data shape: {raw_shape}")
report_lines.append(f"Processed data shape: {processed_shape}\n")
report_lines.append("Missing values before cleaning:\n")
report_lines.append(str(missing_before))
report_lines.append("\nMissing values after cleaning:\n")
report_lines.append(str(missing_after))

# 4️⃣ Save report
report_dir = "reports"
os.makedirs(report_dir, exist_ok=True)
report_path = os.path.join(report_dir, f"cleaning_report_{timestamp}.txt")
with open(report_path, "w") as f:
    f.write("\n".join(report_lines))

print(f"📄 Report saved to: {report_path}")


📄 Report saved to: reports\cleaning_report_20250809_123014.txt
